In [ ]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from sklearn.linear_model import LinearRegression

import voteestimator
import numpy as np

## Read

In [ ]:
class AnalysisSetCreator:
    
    def __init__(self, votesmodel='Meindertsma'):

        votesmodels = {'Meindertsma': voteestimator.MeindertsmaVotesEstimator(),
                      'Exponential': voteestimator.ExponentialVotesEstimator()
                      }
        self.votesmodel = votesmodels[votesmodel]
    
    def _combine_data(self, filefolder):
        self.notering = pd.read_parquet(os.path.join(filefolder, 'notering.parquet'))
        self.song = pd.read_parquet(os.path.join(filefolder, 'song.parquet'))
        self.songartist = pd.read_parquet(os.path.join(filefolder, 'songartist.parquet'))
        self.artist = (pd.read_parquet(os.path.join(filefolder, 'artist.parquet')) # TODO: This should not happen here
                          .pipe(self._artist_features)
                        )
        
        df = (self.notering.merge(self.song, left_on='SongID', right_index=True)
                           .merge(self.songartist.reset_index())
                           .merge(self.artist, left_on='ArtistID', right_index=True, suffixes=('Song', 'Artist'))
             )
        return df
    
    def _read_stemperiodes(self, path=os.path.join('Data', 'EindeStemperiode.xlsx')):
        einde_stemperiode = (pd.read_excel(path, engine='openpyxl')
                               .dropna(subset=['EindeStemperiode'])
                               .drop(columns=['Bron'])
                               .sort_values('EindeStemperiode')
                            )
        return einde_stemperiode
    

    def _check_passed_away_during_top2000(self, df, top2000_stemperiodes):
        first_stemperiode = top2000_stemperiodes['EindeStemperiode'].min()
        relevant_date_of_death = first_stemperiode + pd.Timedelta('365 days')
        df['IsOverleden'] = df['Overlijdensdatum'].ge(relevant_date_of_death)
        return df
    
    def _find_next_top2000_after_death(self, df, top2000_stemperiodes):
        not_passed_away_during_top_2000 = df[~df['IsOverleden']].copy()
        passed_away_during_top2000 = (df.loc[df['IsOverleden']]
                                      .sort_values('Overlijdensdatum')
                                      .reset_index()
                                     )

        passed_away_during_top2000 = (pd.merge_asof(passed_away_during_top2000, top2000_stemperiodes,
                                                   left_on='Overlijdensdatum', right_on='EindeStemperiode', direction='forward')
                                     .set_index('ArtistID')
                                     )
        df = pd.concat([not_passed_away_during_top_2000, passed_away_during_top2000], sort=False)
        return df


    def _artist_features(self, df):
        einde_stemperiode = self._read_stemperiodes()
        df = (df                                          
                .pipe(self._check_passed_away_during_top2000, einde_stemperiode)
                .pipe(self._find_next_top2000_after_death, einde_stemperiode)
                .assign(AgePassing = lambda df: df['Overlijdensdatum'].sub(df['Geboortedatum']).dt.days / 365.25,)
             )
        return df
    
    def _rank_features(self, df):
        return df.assign(PctVotes = lambda df: df['Rank'].apply(self.votesmodel.percentage_of_votes))
    
    
    def _normalize_by_years_before_death(self, df, years_to_normalize=2):
        fill_value = 0.5*self.votesmodel.percentage_of_votes(2000)
        votes_before_death = (df.query('IsOverleden')
                                .pivot_table('PctVotes', index=['SongID', 'ArtistID'], columns=['YearsSinceOverlijden'],
                                             fill_value=fill_value)
                                .loc[:, range(-years_to_normalize, 0)]
                                .mean(axis='columns')
                                .rename('PctVotesBeforeDeath')
                                .reset_index()
                                .fillna(fill_value)
                             )
        df = df.merge(votes_before_death, how='left')
        return df

    
    def _song_features(self, df):
        
        df = (df.assign(NrArtists = lambda df: df.groupby(['SongID', 'Year'])['Rank'].transform('count'),
                        YearsSinceOverlijden = lambda df: df['Year'].sub(df['JaarTop2000']),
                       )
                .pipe(self._normalize_by_years_before_death)
             )
        return df
    
    def _song_features_after_passing(self, df):
        df = (df.assign(NrsBeforeDeath = lambda df: df.groupby('ArtistID')['ArtistID'].transform('count'),
                        PopularityWithinArtist = lambda df: df.groupby('ArtistID')['PctVotesBeforeDeath'].apply(lambda v: v.div(v.mean())),
                        LogSongPopularityWithinArtist = lambda df: np.log10(df['PopularityWithinArtist']),
                        RecencyWithinArtist = lambda df: df.groupby('ArtistID')['YearMade'].apply(lambda v: v.sub(v.min()).div(v.max() - v.min())),
                        YearsBeforeDeath = lambda df: df['YearMade'].sub(df['JaarTop2000']),
                        Boost = lambda df: df['PctVotes'].div(df['PctVotesBeforeDeath']),
                        MultiplePerformers = lambda df: df['NrArtists'].gt(1).astype(int),
                        JarenGeleden = lambda df: df['JaarTop2000'].sub(df['JaarTop2000'].max()),
                        )
             )
        return df
    
    def create_analysis_set(self, filefolder):
        df = (self._combine_data(filefolder)
                  .pipe(self._rank_features)
                  .pipe(self._song_features)
                  .query('YearsSinceOverlijden == 0')
                  .query(f'PctVotesBeforeDeath > {0.5 * self.votesmodel.percentage_of_votes(2000)}')
                  .pipe(self._song_features_after_passing)
             )
        return df
    
    def create_artist_set(self, filefolder):
        df = self.create_analysis_set(filefolder)
        df_artist = (df.groupby('ArtistID')
                        .agg({'PctVotes': 'sum',
                              'PctVotesBeforeDeath': 'sum',
                               'YearMade': 'last'
                            }
                            )
                        .join(self.artist[['Name', 'IsDutch', 'AgePassing', 'JaarTop2000', 'Overlijdensdatum', 'EindeStemperiode']])
                        .assign(DaysToStemperiode = lambda df: df['Overlijdensdatum'].sub(df['EindeStemperiode']).dt.days,
                                YearsSinceLastHit = lambda df: df['JaarTop2000'].sub(df['YearMade']),
                                LogPopularity = lambda df: np.log10(df['PctVotesBeforeDeath']),
                                Boost = lambda df: df['PctVotes'].div(df['PctVotesBeforeDeath']),
                                LogBoost = lambda df: np.log(df['Boost']),
                                )
                    )
        return df_artist
    
    def create_full_feature_set(self, filefolder):
        df = self.create_analysis_set(filefolder)
        df_artist = self.create_artist_set(filefolder)#.pipe(self._artist_features)
        full_set = (df.merge(df_artist, left_on='ArtistID', right_index=True, suffixes=('Song', 'Artist'))
                      .assign(
                              SongRelativeBoost = lambda df: df['BoostSong'].div(df['BoostArtist']),
                              LogRelativeBoost = lambda df: np.log2(df['SongRelativeBoost']),
                              LogBoost = lambda df: np.log(df['BoostSong']),
                             )
           )
        return full_set

a = AnalysisSetCreator()
df_song = a.create_analysis_set('Data')
df_artist = a.create_artist_set('Data')
df = a.create_full_feature_set('Data')

## Create analysis set

## Univariate analysis
* Newsworthyness of artist passing
  * Age when passing away
  * Dutch nationality
  * Popularity
  * Year
  * Recency of hits
* Days to stemperiode

* Song
  * Popularity within artist
  * Recency of song within artist
  
### Newsworthyness

#### Artist age

In [ ]:
def plot_with_trend(df, column, logy=False):
    ycolname = 'LogBoost' if logy else 'Boost'
    preds = LinearRegression().fit(df[[column]], df[ycolname]).predict(df[[column]])
    ax = df.plot(x=column, y=ycolname, kind='scatter')
    ax.plot(df[column], preds, 'k--')
    return ax

In [ ]:
ax = plot_with_trend(df_artist, 'AgePassing', logy=True)

#### Jaar

In [ ]:
ax = plot_with_trend(df_artist, 'JaarTop2000', logy=True)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

#### Nationality

In [ ]:
_ = sns.catplot(x='IsDutch', y='LogBoost', data=df_artist)

#### Popularity

In [ ]:
ax = plot_with_trend(df_artist, 'PctVotes', logy=True)

In [ ]:
plot_with_trend(df_artist, 'LogPopularity', logy=True)

In [ ]:
plot_with_trend(df_artist.query('PctVotesBeforeDeath > 0.00006'), 'LogPopularity', logy=True)

#### Recency

In [ ]:
def recency_effect(x, exponent=1.1, max_effect=2):
    return exponent ** x * max_effect 
ax = plot_with_trend(df_artist.query('DaysToStemperiode > -365'), 'DaysToStemperiode', logy=True)
x = df_artist['DaysToStemperiode'].clip(lower=-365).sort_values().values
# ax.plot(x, recency_effect(x, 1.02, 1))
ax.plot(x, recency_effect(x, 1.07, 1) + 0.4, 'k--')

In [ ]:
recency_buckets = pd.cut(df_artist['DaysToStemperiode'].clip(lower=-365), 6, labels=False, retbins=False, right=False)

In [ ]:
recency_buckets = recency_buckets.map({i: v for i, v in enumerate(range(-333, 0, 60))})

In [ ]:
recency_buckets

In [ ]:
df_artist.groupby(recency_buckets)['LogBoost'].agg(['mean', 'sem', 'std', 'count'])

In [ ]:
ax = df_artist.groupby(recency_buckets)['LogBoost'].agg(['mean', 'sem', 'std', 'count']).plot(y='mean', yerr='sem')
plt.plot(x, recency_effect(x, 1.07, 1)+ 0.4, 'k--')
plt.xlim(-365, 0)
plt.show()

### Recency of last hit

In [ ]:
ax = plot_with_trend(df_artist, 'YearsSinceLastHit', logy=True)

### Song

### Solo song

In [ ]:
df.groupby(['NrArtists'])['LogBoost'].agg(['mean', 'std', 'sem', 'count'])

In [ ]:
df.groupby(['MultiplePerformers'])['LogBoost'].agg(['mean', 'std', 'sem', 'count'])

### Popularity within artist oeuvre

In [ ]:
ax = plot_with_trend(df, 'PopularityWithinArtist', logy=True)

In [ ]:
ax = plot_with_trend(df, 'LogSongPopularityWithinArtist', logy=True)

In [ ]:

df.query('NrsBeforeDeath > 2')

### Recency within artist

In [ ]:
ax = plot_with_trend(df.query('NrsBeforeDeath > 2'), 'RecencyWithinArtist', logy=True)

## Multivariate

In [ ]:
import pymc3 as pm
import arviz as az

In [ ]:
passed_away_artists = df['ArtistID'].unique()
artist_lookup = dict(zip(passed_away_artists, range(len(passed_away_artists))))

In [ ]:
artist_vals = df['ArtistID'].replace(artist_lookup).values

In [ ]:
artist_model = (df.assign(ArtistIDModel = lambda df: df['ArtistID'].map(artist_lookup))
                .sort_values('ArtistIDModel')
                .drop_duplicates(['ArtistIDModel'])
               )

In [ ]:
coords = {"obs_id": np.arange(df.shape[0]), 'Artist': passed_away_artists}
with pm.Model(coords=coords) as multilevel_model:
    artist_idx = pm.Data("artist_idx", artist_vals, dims="obs_id")
    jaren_geleden = pm.Data("jaren_geleden", artist_model['JarenGeleden'], dims='Artist')
    logpopularity = pm.Data('logpopularity', artist_model['LogPopularity'].add(artist_model['LogPopularity'].median()), dims='Artist')
    days_to_stemperiode = pm.Data('days_to_stemperiode', artist_model['DaysToStemperiode'].clip(lower=-365), dims='Artist')
    
    multiple_performers = pm.Data('multiple_performers', df['MultiplePerformers'], dims="obs_id")
    popularity_within_oeuvre = pm.Data('popularity_within_oeuvre', df['LogSongPopularityWithinArtist'], dims="obs_id")
    
    # Hyperpriors:
    a = pm.Normal("a", mu=0, sigma=10.0)
    sigma_a = pm.Exponential("sigma_a", 1.0)
    recency_effect_exponent = 1 + pm.Exponential('recency_effect_exponent', 5)
    max_recency_effect = pm.Exponential('max_recency_effect', 1)

    effect_popularity = pm.Normal('effect_popularity', mu=0, sigma=10)
    history_effect = pm.Normal('history_effect', mu=0, sigma=10)
    
    mu_artist = (a
                 + effect_popularity * logpopularity
                 + recency_effect_exponent ** days_to_stemperiode * max_recency_effect
                 + jaren_geleden * history_effect)
    
    a_artist = pm.Normal("a_artist", mu=mu_artist, sigma=sigma_a, dims="Artist")

    # Expected value per artist:
    sharing_effect = pm.Normal('sharing_effect', mu=0, sigma=10.0)
    within_oeuvre_effect = pm.Normal('within_oeuvre_effect', mu=0, sigma=10.0)
    theta = (a_artist[artist_idx]
             + multiple_performers * sharing_effect
             + within_oeuvre_effect * popularity_within_oeuvre
            )
    # Model error:
    sigma = pm.Exponential("sigma", 1.0)

    y = pm.Normal("y", theta, sigma=sigma, observed=df['LogBoost'], dims="obs_id")
pm.model_to_graphviz(multilevel_model)

In [ ]:
RANDOM_SEED = 123
with multilevel_model:
    multilevel_model_idata = pm.sample(5000, tune=2000, return_inferencedata=True, random_seed=RANDOM_SEED, target_accept=0.95)


In [ ]:
var_names = ["a", #"sigma_a", "sigma",
             'effect_popularity',
             'max_recency_effect',
             'recency_effect_exponent',
             'history_effect',
             'sharing_effect',
             'within_oeuvre_effect'
            ]

In [ ]:
az.summary(multilevel_model_idata, var_names=var_names + ['sigma_a', 'sigma'], round_to=2)

In [ ]:
_ = pm.pairplot(multilevel_model_idata, var_names=var_names, marginals=True)

In [ ]:
_ = az.plot_trace(multilevel_model_idata, compact=True)

In [ ]:
with multilevel_model:
    ppc = pm.sample_posterior_predictive(
        multilevel_model_idata, random_seed=RANDOM_SEED, var_names=var_names+['y']
    )


In [ ]:
parameters = pd.DataFrame({k: v for k, v in ppc.items() if k != 'y'})

In [ ]:
parameters.corr()

In [ ]:
outcomes = {}
days = np.arange(-365, 0)
for i, row in parameters.iterrows():

    recencyeffect = (1 + row['recency_effect_exponent'])  ** days * row['max_recency_effect']
    outcomes[i] = recencyeffect

In [ ]:
df_recencyeffect = pd.DataFrame(outcomes, index=np.arange(-365, 0))

In [ ]:
%%time
ax = df_recencyeffect.iloc[:, :2500].plot(c='grey', alpha=0.01, legend=False)
ax = df_recencyeffect.median(axis='columns').plot(ax=ax, lw=2)
plt.show()

In [ ]:
df_recencyeffect.transpose().quantile([0.025, 0.16, 0.5, 0.84, 0.975]).transpose().tail()#plot()

In [ ]:
predictions = pd.DataFrame(ppc['y'].T, index=df.index)

In [ ]:
df['LogBoost']

In [ ]:
trala = pd.concat([predictions.quantile([0.16, 0.5, 0.84], axis='columns').transpose(), df['LogBoost']], axis='columns')

In [ ]:
trala

In [ ]:
plt.errorbar(trala[0.5], trala['LogBoost'], yerr=[trala[0.5].sub(trala[0.16]), trala[0.84].sub(trala[0.5])], ls=' ', marker='o', alpha=0.2, ms=4)
plt.plot([0, 2], [0, 2], 'k--')


In [ ]:
predictions.std(axis='columns').describe()

In [ ]:
posterior = df.assign(yhat = predictions.mean(axis='columns'),
                      yhat_rescaled = lambda df: -0.14 + 1.16 * df['yhat'],
                     error = lambda df: df['yhat'].sub(df['LogBoost']),
                     abserror = lambda df: df['error'].abs(),
                      error_rescaled = lambda df: df['yhat_rescaled'].sub(df['LogBoost']),
                    abserror_rescaled = lambda df: df['error_rescaled'].abs(),
                     )

In [ ]:
posterior['abserror'].mean()

In [ ]:
posterior.nlargest(10, 'abserror')

In [ ]:
posterior[['abserror', 'abserror_rescaled']].describe()

In [ ]:
posterior['error_rescaled'].hist(bins=25)

In [ ]:
posterior[['error'].hist(bins=25)

In [ ]:
posterior.nlargest(10, 'error')#sort_values('LogBoost', ascending=False)

In [ ]:
%matplotlib inline
ax = pd.concat([df['LogBoost'], predictions.mean(axis='columns').rename('yhat')], axis='columns').plot(y='LogBoost', x='yhat', kind='scatter')
ax.plot([0, 3], [0, 3], 'k--')

In [ ]:
posterior['abserror'].mean()

In [ ]:
posterior[['LogBoost', 'yhat']].corr()

In [ ]:
ppc['y'].mean(axis=0)

In [ ]:
az.from_pymc3(posterior_predictive=ppc, model=multilevel_model)

In [ ]:
az.plot_ppc(az.from_pymc3(posterior_predictive=ppc, model=multilevel_model));